# PydanticOutputParser

`PydanticOutputParser`는 언어 모델의 출력을 **구조화된 정보**로 변환하는 데 도움을 주는 클래스이다. 
이 클래스는 단순 텍스트 응답 대신 **명확하고 체계적인 형태로 필요한 정보를 제공**할 수 있다.

이 클래스를 활용하면 언어 모델의 출력을 특정 데이터 모델에 맞게 변환하여 정보를 더 쉽게 처리하고 활용할 수 있다.

In [1]:

from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser

In [2]:
from llm_provider import get_llm
# llm = get_llm(provider="ollama",model="llama3.2")
llm = get_llm()

In [3]:
def stream_res(response):
    for token in response:
        print(token, end="", flush=True)

In [4]:
email_conversation = """
From: 박철수 (busi4876@kakao.com)
To: OSC 담당자 (sales@osckorea.com)

Subject: 2025년 중소기업 정책자금 융자 신청 안내

안녕하세요. 중소기업 지원센터 박철수 팀장입니다.
2024년도 하반기 정책 자금 신청의 경우 잔여 예산 대비 과도한 경쟁률로 인해 심사 지연 혹은 재심사되는 경우가 다수 발생되었습니다.
그렇기에 자금이 풀리는 2025년의 상반기에 빠르게 신청하셔야 합니다. [상반기에 해당되는 5월 이후에는 자금 수령의 어려움이 있습니다. ]
장기간 쌓여온 노하우를 바탕으로 신청부터 지급까지 체계적으로 도와드리겠습니다.
신청 및 안내를 원하실 경우 아래의 연락처로 연락 바랍니다

1. 2025년 상반기 정책자금 예정
- 5차 : 2025년5월초 (예정)

2. 정책자금 신청 시 전문가 필요성

ㅇ 2025년 정책자금 중소기업 융자 4조 5280억원, 이차보전 6,027억원 투입
2024년과 비교시 융자 지원금이 낮아졌습니다. 시간이 지날수록 융자의 비율이 줄어들 수 있으므로, 가능한 빠르게 준비하셔야 합니다.

위와 관련해서  협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일 오전 10시에 미팅을 제안합니다. 
귀사 사무실에서 만나 이야기를 나눌 수 있을까요?


박철수 
이사
중소기업 지원센터
"""

In [5]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

chain = prompt | llm | StrOutputParser()

answer = chain.stream({"email_conversation": email_conversation})

stream_res(answer)


중요한 내용 요약:

1. **정책자금 신청 시기**: 2025년 상반기(예정: 5차, 2025년 5월 초)에 신청해야 함.
2. **경쟁률**: 2024년도 하반기 정책 자금 신청 시 잔여 예산 대비 과도한 경쟁률로 심사 지연 및 재심사 사례 발생.
3. **지원 금액**: 2025년 정책자금으로 중소기업 융자 4조 5280억원, 이차보전 6,027억원 투입 예정. 2024년과 비교하여 지원금이 낮아짐.
4. **전문가 필요성**: 신청 및 지급 과정에 전문가의 도움이 필요함. 신속히 준비해야 함.
5. **미팅 제안**: 다음 주 화요일 오전 10시에 귀사 사무실에서 미팅 제안. 협력 가능성 논의 예정.

연락처를 통해 신청 및 안내 요청 필요.

In [6]:
## Pydantic 스타일로 정의된 클래스를 사용하여 이메일의 정보를 파싱한다
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [7]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

In [8]:

chain = prompt | llm | parser

In [9]:
# chain 을 실행하고 결과를 출력.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력
response

EmailSummary(person='박철수', email='busi4876@kakao.com', subject='2025년 중소기업 정책자금 융자 신청 안내', summary='2025년 상반기 정책자금 신청의 필요성을 강조하며, 전문가의 도움이 필요하다는 내용을 포함하고 있습니다. 2025년 5월 초에 융자 신청 예정이며, 다음 주 화요일 오전 10시에 미팅 제안을 하고 있습니다.', date='다음 주 화요일 오전 10시')

## with_structured_output()

In [10]:
llm_with_structered = llm.with_structured_output(EmailSummary)

In [11]:
# invoke() 함수를 호출하여 결과를 출력
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='박철수', email='busi4876@kakao.com', subject='2025년 중소기업 정책자금 융자 신청 안내', summary='박철수 팀장은 2024년 하반기 정책 자금 신청의 경쟁률과 심사 지연 문제를 언급하며, 2025년 상반기에 빠르게 신청할 것을 권장했습니다. 특히 5월 이후 자금 수령이 어려울 수 있으며, 신청에서 지급까지 지원할 수 있다고 했습니다. 또한 2025년 정책자금이 4조 5280억원으로 예상되며, 지원금이 줄어들 수 있기에 빠른 준비가 필요하다고 강조했습니다. 다음 주 화요일 오전 10시에 미팅을 요청했습니다.', date='2023-10-17')

In [12]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | llm | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 온실가스의 증가로 인해 발생합니다. 이로 인해 기후 변화, 극단적인 날씨, 해수면 상승 등의 문제가 생길 수 있습니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#지구']}
